# PS7
## Li Ruixue
### 1. Classifier "horse" race
#### (a) Use sklearn.linear model.LogisticRegression to fit a logistic model of mpg high on features number of cylinders (cyl), engine displacement (dspl), horsepower (hpwr), vehicle weight (wgt), acceleration (accl), vehicle year (yr), vehicle origin (orgn). Make sure to include a constant term. Fit the model using k-fold cross validation with k = 4 folds.2 kf_log = KFold(n_splits=4, shuffle=True, random_state=15) Report the MSE of the model as the average MSE across the k = 4 test sets, and report the error rates for each category of mpg high as the average error rate for that category across the k = 4 test sets.

In [113]:
import numpy as np
from sklearn import svm

import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report, mean_squared_error
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold

%matplotlib inline


In [114]:
df = pd.read_csv("../../ProblemSets/PS7/Auto.csv", na_values="?")
df.isnull().sum()

mpg             0
cylinders       0
displacement    0
horsepower      5
weight          0
acceleration    0
year            0
origin          0
name            0
dtype: int64

In [115]:
df = df.dropna()
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


In [116]:
m = df['mpg'].median()
df['mpg_high'] = np.where(df['mpg'] > m, 1,0)
df['const'] = 1

In [117]:
Xvars = df[['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'year', 'origin']].values
yvals = df['mpg_high'].values
print("Since the type of error is not specified, I report the error rate as : the rate\
 of false negatives (1-recall) \n \
      ")
k = 4
kf = KFold(n_splits=k, random_state=15, shuffle=True)
kf.get_n_splits(Xvars)

MSE_vec_kf = np.zeros(k)
err_low_vec = np.zeros(k)
err_high_vec = np.zeros(k)
err_low_vec = np.zeros(k)
err_high_vec = np.zeros(k)
y_pred_vec = np.array([])
y_test_vec = np.array([])

k_ind = int(0)
for train_index, test_index in kf.split(Xvars):
    print('\n\n')
    print('Group ', k_ind)
    X_train, X_test = Xvars[train_index], Xvars[test_index]
    y_train, y_test = yvals[train_index], yvals[test_index]
    LogReg = LogisticRegression(fit_intercept=True)
    LogReg.fit(X_train, y_train)
    y_pred = LogReg.predict(X_test)
    y_pred_vec = np.append(y_pred_vec, y_pred)
    y_test_vec = np.append(y_test_vec, y_test)
    err_low = ((y_test==0)*(y_test!=y_pred)).sum() / (y_test==0).sum()
    err_high = ((y_test==1)*(y_test!=y_pred)).sum() / (y_test==1).sum()
    MSE_vec_kf[k_ind] = ((y_test - y_pred) ** 2).mean()
    err_low_vec[k_ind] = err_low
    err_high_vec[k_ind] = err_high
    print(classification_report(y_test, y_pred))
    print('MSE for test set', k_ind, ' is', MSE_vec_kf[k_ind])
    print('Error rate for mpg_high = 0:', err_low)
    print('Error rate for mpg_high = 1:', err_high)
    k_ind += 1

MSE_kf = MSE_vec_kf.mean()

err_low_avg = err_low_vec.mean()
err_high_avg = err_high_vec.mean()
    
print('\n\n')
print('Full data set:')
print(classification_report(y_test_vec, y_pred_vec))
print('Model MSE:', MSE_kf)
print('Error rate for mpg_high = 0:', err_low_avg)
print('Error rate for mpg_high = 1:', err_high_avg)

Since the type of error is not specified, I report the error rate as : the rate of false negatives (1-recall) 
       



Group  0
             precision    recall  f1-score   support

          0       0.94      0.89      0.92        55
          1       0.87      0.93      0.90        43

avg / total       0.91      0.91      0.91        98

MSE for test set 0  is 0.09183673469387756
Error rate for mpg_high = 0: 0.10909090909090909
Error rate for mpg_high = 1: 0.06976744186046512



Group  1
             precision    recall  f1-score   support

          0       0.88      0.91      0.90        47
          1       0.92      0.88      0.90        51

avg / total       0.90      0.90      0.90        98

MSE for test set 1  is 0.10204081632653061
Error rate for mpg_high = 0: 0.0851063829787234
Error rate for mpg_high = 1: 0.11764705882352941



Group  2
             precision    recall  f1-score   support

          0       0.85      0.87      0.86        45
          1       0.88     

#### (b) Use sklearn.ensemble.RandomForestClassifier to fit a random forest model of mpg high on max features=2 out of the seven possible features used in part (a). Set n estimators=20, set bootstrap=True, set oob score=True, and set random state=25. Report the MSE of the random forest model as the MSE from the .oob prediction object, and report the error rates for each category of mpg high from the .oob prediction object.

In [118]:
from sklearn.ensemble import RandomForestClassifier
Xvars_rf = df[['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'year', 'origin']].values
yvals_rf = df['mpg_high'].values

random_forest = RandomForestClassifier(n_estimators=20, max_features=2, bootstrap=True,
                                  oob_score=True, random_state=25)
random_forest.fit(Xvars_rf, yvals_rf)

random_forest.score(Xvars_rf, yvals_rf)
MSE_df = pd.DataFrame({'ypred': random_forest.oob_decision_function_.T[1], 'y': yvals})
MSE_df['ypred'] = MSE_df['ypred'].apply(lambda x: 1 if int(x>=0.5) else 0)
MSE = mean_squared_error(MSE_df['y'], MSE_df['ypred'])
    
y_pred = np.array(MSE_df['ypred'])
y_test = np.array(yvals_rf)
err_low = ((y_test==0)*(np.abs(y_test-y_pred))).sum() / (y_test==0).sum()
err_high = ((y_test==1)*(np.abs(y_test-y_pred))).sum() / (y_test==1).sum()
    
print('Model MSE=', MSE)
print('Error rate for mpg_high = 0:', err_low)
print('Error rate for mpg_high = 1:', err_high)

Model MSE= 0.07142857142857142
Error rate for mpg_high = 0: 0.08673469387755102
Error rate for mpg_high = 1: 0.05612244897959184


#### (c) Use sklearn.svm.SVC to fit a support vector machines model of mpg high with a Gaussian radial basis function kernel kernel='rbf' on the seven features used in part (a). Set the penalty parameter to C=1 and set gamma=0.2. Fit the model using k-fold cross validation with k = 4 folds exactly as in part (a). Report the MSE of the model as the average MSE across the k = 4 test sets, and report the error rates for each category of mpg high as the average error rate for that category across the k = 4 test sets.

In [119]:
Xvars = df[['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'year', 'origin']].values
yvals = df['mpg_high'].values
k = 4
kf_svm = KFold(n_splits=k, random_state=15, shuffle=True)
kf.get_n_splits(Xvars)

MSE_vec_kf = np.zeros(k)
y_pred_vec = np.array([])
y_test_vec = np.array([])
err_low_vec = np.zeros(k)
err_high_vec = np.zeros(k)

k_ind = int(0)
for train_index, test_index in kf.split(Xvars):
    print('\n\n')
    print('Group ', k_ind)
    X_train, X_test = Xvars[train_index], Xvars[test_index]
    y_train, y_test = yvals[train_index], yvals[test_index]
    
    auto_svm = svm.SVC(kernel='rbf', gamma=0.2, C=1)
    auto_svm.fit(X_train, y_train)
    
    y_pred = auto_svm.predict(X_test)
    y_pred_vec = np.append(y_pred_vec, y_pred)
    y_test_vec = np.append(y_test_vec, y_test)
    
    err_low = ((y_test==0)*(np.abs(y_test-y_pred))).sum() / (y_test==0).sum()
    err_high = ((y_test==1)*(np.abs(y_test-y_pred))).sum() / (y_test==1).sum()
    err_low_vec[k_ind] = err_low
    err_high_vec[k_ind] = err_high
    MSE_vec_kf[k_ind] = ((y_test != y_pred).astype(int)).mean()
    print(classification_report(y_test, y_pred))
    print('MSE for test set', k_ind, ' is', MSE_vec_kf[k_ind])
    print('Error rate for mpg_high = 0:', err_low)
    print('Error rate for mpg_high = 1:', err_high)
    k_ind += 1

MSE_kf = MSE_vec_kf.mean()

err_low_avg = err_low_vec.mean()
err_high_avg = err_high_vec.mean()
    
print('\n\n')
print('Full data set:')
print(classification_report(y_test_vec, y_pred_vec))
print('Model MSE:', MSE_kf)
print('Error rate for mpg_high = 0:', err_low_avg)
print('Error rate for mpg_high = 1:', err_high_avg)




Group  0
             precision    recall  f1-score   support

          0       1.00      0.04      0.07        55
          1       0.45      1.00      0.62        43

avg / total       0.76      0.46      0.31        98

MSE for test set 0  is 0.5408163265306123
Error rate for mpg_high = 0: 0.9636363636363636
Error rate for mpg_high = 1: 0.0



Group  1
             precision    recall  f1-score   support

          0       0.48      1.00      0.65        47
          1       0.00      0.00      0.00        51

avg / total       0.23      0.48      0.31        98

MSE for test set 1  is 0.5204081632653061
Error rate for mpg_high = 0: 0.0
Error rate for mpg_high = 1: 1.0



Group  2
             precision    recall  f1-score   support

          0       0.47      1.00      0.64        45
          1       1.00      0.04      0.07        53

avg / total       0.76      0.48      0.33        98

MSE for test set 2  is 0.5204081632653061
Error rate for mpg_high = 0: 0.0
Error rate fo

/Users/liruixue/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


#### (d) Which of the above three models do you think is the best predictor of mpg_high? Why?

Random forest, since it has smallest MSE and error rate of each category.